### Only Run on Google Colab

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
%cd /content/drive/MyDrive/github/FYP_low_light_image_enhancement/

/content/drive/MyDrive/github/FYP_low_light_image_enhancement


In [12]:
%ls

main.py  notebook.ipynb


# Low Light Image Enhancement

In [13]:
print("Hello world")

Hello world


In [14]:
print("hi")

hi


In [15]:
from main import Sample

In [16]:
sample = Sample()

In [17]:
sample.name

'test'

In [18]:
sample.printName()

My name is test
